### Importing Packages

In [ ]:
# Standard data science libraries
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')

# Plotly 
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

### Building Energy Data

For these plots, we will be examining building energy data. We have data for one building, with 3 different types of sensors: Energy, Steam, and Static Pressure. This dataframe has a multiindex on the columns which is a helpful way to keep track of hierarchical relationships. 

In [ ]:
# Read in data and convert index to a datetime
df = pd.read_csv('building_one.csv')
df.head()

In [ ]:
df.drop(index = [0, 1], inplace = True)
df = df.reset_index(drop= True)

In [ ]:
df = df.set_index('type')

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
df

In [ ]:
energy_series = df['Energy']

Let's see what matplotlib can do with this.

In [ ]:
energy_series.plot(figsize = (10, 8));
plt.ylabel('kwh'); plt.title('Energy Usage');

# Basic Time Series Plot

We will being by just making the most introductory plot we can. This requires a little more code than in matplotlib, but it gives us more features, such as interactivity with no extra effort.

First, we need to create a data object. We use `go.Scatter` and provide it with x and y values. By default, with more than 20 points, the mode will be set to `lines`. Here we'll just use the defaults for all of the parameters.

In [ ]:
energy_data = go.Scatter(x=energy_series.index,
                        y=energy_series.values)

Then, we set up the plot layout. This will set the defaults again although we do provide the minimum labels.

In [ ]:
layout = go.Layout(title='Energy Plot', xaxis=dict(title='Date'),
                   yaxis=dict(title='(kWh)'))

Now, we just need to create a figure and display it. When we pass in the data, it needs to be a list even with a single item.

In [ ]:
fig = go.Figure(data=[energy_data],layout=layout)

Finally, we can display the plot interactively in the notebook.

In [ ]:
iplot(fig)

## Plot Formatting

This plot is pretty impressive by itself (better than matplotlib), and we can add a few parameters to make things look better than the default. Below we format the hover information using `text`, set the `line` parameters, adjust the `height` and `width` of the plot, and show the results.

In [ ]:
# Create the data object
energy_data = go.Scatter(x=energy_series.index,
                        y=energy_series.values,
                        line=go.scatter.Line(color='red', width = 0.8),
                           opacity=0.8,
                           name='Energy',
                           text=[f'Energy: {x:.0f} kWh' for x in energy_series.values])


# Set up the plot layout
layout = go.Layout(height=800, width=800, titlefont=dict(size=20),
                   font=dict(size=16),
                   title='Energy Plot', xaxis=dict(title='Date'),
                   yaxis=dict(title='(kWh)'))

# Create figure and display
fig = go.Figure(data=[energy_data], layout=layout)
iplot(fig)

# Secondary Y-Axis

If we want to include multiple data sources with different ranges, then we will need to use a secondary yaxis. Fortunately, this is simple in plotly and we'll just add it to the interactive plot. 

In [ ]:
steam_series = df["Steam"]
steam_series

The key is to add the secondary data source using a `yaxis` of `y2`. This plots the data on the second yaxis which will add.

In [ ]:
steam_data = go.Scatter(x=steam_series.index,
                        y=steam_series.values,
                        line=dict(color='blue', width=0.8),
                        opacity=0.8,
                        name='Steam',
                        yaxis='y2',
                        text=[f'Steam: {x:.1f} Mlbs/hr' for x in steam_series.values])

### Add to Layout

We also need to add a secondary `yaxis` to the layout.

In [ ]:
# Create a layout with two yaxes
layout = go.Layout(height=600, width=1000, font=dict(size=18),
                   title='Energy and Steam Plot',
                   xaxis=dict(title='Date', type='date'),
                   
                   yaxis=dict(title='Energy', color='red'),
                   # Add a second yaxis to the right of the plot
                   yaxis2=dict(title='Steam', color='blue',
                                          overlaying='y',
                                          side='right')
                   )

fig = go.Figure(data=[energy_data, steam_data], layout=layout)
iplot(fig)

There it is! We have the energy and the steam on a single plot. We can see that steam is only used during the winter (for heating) while of course we need energy year-round! 

# Annotations

Annotations allow us to call our (highlight) aspects of our data. Let's look at how to annotate a weeks worth of steam measurements with the daily high values.

First, we'll get a weeks worth of steam data.

In [ ]:
df_short = df.loc[df.index.week == 6].copy()
df_short.head()
df_short.tail()

In [ ]:
steam_series_four = df_short['Steam']
steam_series_five = df_short['Steam.1']

These can be made into data objects as follows:

In [ ]:
steam_data_four = go.Scatter(
    x=steam_series_four.index,
    y=steam_series_four.values,
    line=dict(color='blue', width=1.1),
    opacity=0.8,
    name='Steam: Sensor 4',
    hoverinfo = 'text',
    text = [f'Sensor 4: {x:.1f} Mlbs/hr' for x in steam_series_four.values])

In [ ]:
steam_data_five = go.Scatter(
    x=steam_series_five.index,
    y=steam_series_five.values,
    line=dict(color='orange', width=1.1),
    opacity=0.8,
    name='Steam: Sensor 5',
    hoverinfo='text',
    text=[f'Sensor 5: {x:.1f} Mlbs/hr' for x in steam_series_five.values])

Then, we'll write a short function to find the daily maximumn steam measurement.

In [ ]:
def find_daily_maxes(x):
    """Return maximum measurement on each day and when it occurred in a dataframe"""
    x = x.copy().to_frame()
    x['day'] = x.index.day
    result =pd.concat([x.groupby('day').max(), 
                      x.groupby('day').idxmax()], axis = 1).iloc[:, [0, 1]]
    result.columns = ['value', 'date']
    return result.set_index('date')

four_highs = find_daily_maxes(steam_series_four)
five_highs = find_daily_maxes(steam_series_five)
four_highs

We'll also use a short function to format the time of day at which the maximum occurred.

In [ ]:
from datetime import datetime

def format_time(dt):
    if pd.isnull(dt):
        return "NaT"
    else:
        return datetime.strftime(dt, "%a <br> %H:%M %p")
    
format_time(datetime.now())

## Building Annotations

The following creates the annotation using a list comprehension. For each annotation, we specify the x and y position, along with the formatted text to display. We can adjust other aspects, but we'll let plotly take care of those details. 

In [ ]:
four_annotations = [dict(x = date, y = value[0], 
                         xref = 'x', yref = 'y', 
                         font=dict(color = 'blue'),
                         text = f'{format_time(date)}<br> {value[0]:.1f} Mlbs/hr')
                    for date, value in zip(four_highs.index, four_highs.values)]

In [ ]:
four_annotations[:2]

In [ ]:
five_annotations = [dict(x = date, y = value[0], xref = 'x', yref = 'y', font=dict(color = 'orange'),
                         text = f'{format_time(date)}<br> {value[0]:.1f} Mlbs/hr')
                    for date, value in zip(five_highs.index, five_highs.values)]

### Add Annotations to Plot

To actually show the annotations on the plot, all we need is to add them to the layout. Then we display the plot as before.

In [ ]:
layout = go.Layout(height=800, width=1000, 
                   title='Steam Sensor with Daily High Annotations',
                   annotations=four_annotations)

fig = go.Figure(data = [steam_data_four],
                layout=layout)
iplot(fig)

# Range Selection

This plot already has some basic interaction, but we can improve that by adding a range slider. This will allow us to choose the timescale and the frame of time.

We'll go back to the plot showing energy to demonstrate range selection. The data object will stay the same, but the layout needs to be modified.

## Range Selector and Range Slider

In particular, we will set the xaxis to have a `rangeselector`, which we use for selecting the time scale, and a `rangeslider` which is used to select the window of time. For the `rangeselector`, we add in buttons that allow us select the time scale. The `rangeslider` is easier to code because it does not have buttons. The yaxis remains the same.

For more information, see the documentation on [range selector](https://plot.ly/python/reference/#layout-xaxis-rangeselector) 
and [range slider](https://plot.ly/python/reference/#layout-xaxis-rangeslider). You can also look [this example](https://plot.ly/python/range-slider/).

In [ ]:
# Create the same data object
energy_data = go.Scatter(x=energy_series.index,
                        y=energy_series.values,
                        line=go.scatter.Line(color='red', width = 0.6),
                           opacity=0.8,
                           name='energy',
                           text=[f'energy: {x:.0f} kWh' for x in energy_series.values])


# Create a layout with a rangeselector and rangeslider on the xaxis
layout = go.Layout(height=600, width=900, font=dict(size=18),
                   title='Energy Plot with Range Selection',
                   xaxis=dict(title='Date',
                                        # Range selector with buttons
                                         rangeselector=dict(
                                             # Buttons for selecting time scale
                                             buttons=list([
                                                 # 1 month
                                                 dict(count=1,
                                                      label='1m',
                                                      step='month',
                                                      stepmode='backward'),
                                                 # 1 week
                                                 dict(count=7,
                                                      label='1w',
                                                      step='day',
                                                      stepmode='todate'),
                                                 # 1 day
                                                 dict(count=1,
                                                      label='1d',
                                                      step='day',
                                                      stepmode='todate'),
                                                 # 12 hours
                                                 dict(count=12,
                                                      label='12h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                 # 4 hours
                                                 dict(count=4,
                                                      label='4h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                 # Entire scale
                                                 dict(step='all')
                                             ])
                                         ),
                                         # Sliding for selecting time window
                                         rangeslider=dict(visible=True),
                                         # Type of xaxis
                                         type='date'),
                   # yaxis is unchanged
                   yaxis=dict(title='Static Pressure')
                   )

# Create the figure and display
fig = go.Figure(data=[energy_data], layout=layout)
iplot(fig)

## Range Selector with Double Axes

In [ ]:
# Create a layout with interactive elements and two yaxes
layout = go.Layout(height=600, width=800, font=dict(size=14),
                   title='Energy Plot with Range Selection',
                   xaxis=dict(title='Date',
                                        # Range selector with buttons
                                         rangeselector=dict(
                                             # Buttons for selecting time scale
                                             buttons=list([
                                                 # 1 month
                                                 dict(count=1,
                                                      label='1m',
                                                      step='month',
                                                      stepmode='backward'),
                                                 # 1 week
                                                 dict(count=7,
                                                      label='1w',
                                                      step='day',
                                                      stepmode='todate'),
                                                 # 1 day
                                                 dict(count=1,
                                                      label='1d',
                                                      step='day',
                                                      stepmode='todate'),
                                                 # 12 hours
                                                 dict(count=12,
                                                      label='12h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                 # 4 hours
                                                 dict(count=4,
                                                      label='4h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                 # Entire scale
                                                 dict(step='all')
                                             ])
                                         ),
                                         # Sliding for selecting time window
                                         rangeslider=dict(visible=True),
                                         # Type of xaxis
                                         type='date'),
                   yaxis=dict(title='Energy (kWh)', color='red'),
                   # Add a second yaxis to the right of the plot
                   yaxis2=dict(title='Steam (Mlbs/hr)', color='blue',
                                          overlaying='y',
                                          side='right')
                   )

fig = go.Figure(data=[energy_data, steam_data], layout=layout)
iplot(fig)

# Dropdown Menu

Another option for adding interaction is through a dropdown menu. Here, we will plot a single week and give users the options of which sensor to view for steam. Adding in dropdown menus is a little trickier, but still not too much work for a nice interactive plot! 

### Update Menus

To add the dropdown, we want to create an updatemenu object. This includes the buttons as well as the actions they trigger which are stored in `args`. Because we are changing the visible data, we set the `visible` attribute. We also change the `title` to reflect which sensor(s) is currently shown. 

In [ ]:
updatemenus = list([
    dict(
        active=2,
        buttons=list([
            dict(
                label='Sensor 4',
                method='update',
                args=[{
                    'visible': [True, False]
                }, {
                    'title': 'Sensor 4'
                }]),
            dict(
                label='Sensor 5',
                method='update',
                args=[{
                    'visible': [False, True]
                }, {
                    'title': 'Sensor 5'
                }]),
            dict(
                label='Both',
                method='update',
                args=[{
                    'visible': [True, True]
                }, {
                    'title': 'Sensor Sensors'
                }])
        ]),
    )
])

layout = go.Layout(height=800, width=1000, title='Steam Sensors',
                   updatemenus=updatemenus)

In [ ]:
fig = go.Figure(data=[steam_data_four, steam_data_five], layout=layout)
iplot(fig)

## Dropdown Menu with Annotations

We can easily add in the annotations to the plot by including them in the `args` for the dropdown menu. This will show annotations when we are looking at one or the other sensor but not both.

In [ ]:
updatemenus = list([
    dict(
        active=2,
        buttons=list([
            dict(
                label='Sensor 4',
                method='update',
                args=[{
                    'visible': [True, False]
                }, {
                    'title': 'Sensor 4',
                    'annotations': four_annotations
                }]),
            dict(
                label='Sensor 5',
                method='update',
                args=[{
                    'visible': [False, True]
                }, {
                    'title': 'Sensor 5',
                    'annotations': five_annotations
                }]),
            dict(
                label='Both',
                method='update',
                args=[{
                    'visible': [True, True]
                }, {
                    'title': 'Sensor Sensors'
                }])
        ]),
    )
])

layout = go.Layout(height=600, width=1200, title='Steam Sensors',
                   updatemenus=updatemenus)

In [ ]:
fig = go.Figure(data = [steam_data_four, steam_data_five], 
                layout=layout)

iplot(fig)

# Conclusions

In this notebook, we got an introduction to making time-series plots with plotly. We were able to accomplish quite a lot, even without getting into all the details. This is because plotly is smart enough to interpret our data and figure out the details without our input. Ultimately, this makes us much more productive as data scientists because we can spend more time investigating the data and less time figuring out the particulars of a plot. 

There are many more capabilities in plotly, so check out [the documentation](https://plot.ly/python/) for more. [Dash](https://dash.plot.ly/), created by plotly, can even be used to build web applications with Python code.  